# Temporal Conversions

In [49]:
import pystare
import datetime
import numpy
#import astropy.time

from astropy import time

## Full circle

In [57]:
# Create the TIVs
times = numpy.array(['2021-09-03', '2021-07-17 11:16'], dtype='datetime64')
times = time.Time(times, scale='utc', format='datetime64')
tivs = pystare.from_julian_date(times.jd1, times.jd2, 
                                scale='utc', 
                                forward_res=48, reverse_res=48)
tivs

array([2276059438861267137, 2275939265676325057])

In [55]:
# Back via JD
jd = pystare.to_julian_date(tivs, scale='utc')
times = astropy.time.Time(val=jd[0], val2=jd[1], format='jd')
times.to_datetime()

array([datetime.datetime(2021, 9, 3, 0, 0),
       datetime.datetime(2021, 7, 17, 11, 16)], dtype=object)

In [56]:
# Back via MS
mss = pystare.to_ms_since_epoch_utc(tivs)
dts = [datetime.datetime.utcfromtimestamp(ms/1000) for ms in mss]
[dt.isoformat() for dt in dts]

['2021-09-03T00:00:00', '2021-07-17T11:16:00']

## Valudations

In [7]:
pystare.validate_stare_timestring('2021-01-09T17:47:56.154 (45 12) (1)')

True

In [8]:
pystare.validate_iso8601_string('2021-01-09T17:47:56.2435+05:00')

True

### UTC MS coversions

In [9]:
now = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
now = numpy.array([now], dtype='datetime64[ms]')
now = now.astype(numpy.int64)
now

array([1633457355000])

In [10]:
timestamps = numpy.array(['2021-01-03'], dtype='datetime64[ms]').astype(numpy.int64)
pystare.from_ms_since_epoch_utc(timestamps, 48, 48)

array([2275448110396223681])

In [11]:
tivs = [2275448110396223681]
ts = pystare.to_ms_since_epoch_utc(tivs).astype('datetime64[ms]')
numpy.datetime_as_string(ts)

array(['2021-01-03T00:00:00.000'], dtype='<U42')

In [12]:
ts1 = numpy.array(['2021-01-03T01', '1985-01-03T01'], dtype='datetime64[ms]').astype(numpy.int64)
ts2 = numpy.array(['2021-05-01T10', '1986-10-01'], dtype='datetime64[ms]').astype(numpy.int64)
tiv1 = pystare.from_ms_since_epoch_utc(ts1, 10, 10)
tiv2 = pystare.from_ms_since_epoch_utc(ts2, 10, 10)
pystare.cmp_temporal(tiv1, tiv2, flatten=False)

array([[1, 0],
       [0, 1]])

### Resolutions

In [15]:
millisecond  = 1
second = 1000 * millisecond
minute = 60 * second
hour = 60 * minute
day = 86400 * second
year = 365 * day
times = numpy.array([millisecond, second, minute, hour, day, year], dtype=numpy.int64)
pystare.coarsest_resolution_finer_or_equal_ms(times)

array([48, 38, 32, 26, 21, 12])

In [16]:
pystare.reverse_resolution(tiv1)
pystare.set_reverse_resolution(tiv1, numpy.array([9, 20]))

pystare.lower_bound_tai(tiv1)

array([2270946778511573245, 2230414381613580541])

### Iso Strings

In [18]:
time_strings = ['2021-08-26T17:03:56.6']
pystare.from_iso_strings(time_strings, forward_res=20, reverse_res=18)

array([2276038620409631817])

In [19]:
time_strings = ['2021-08-26T17:03:56.626 (48 48) (1)']
tiv = pystare.from_stare_timestrings(time_strings, scale='TAI')
tiv

array([2276038620410065089])

In [20]:
pystare.to_stare_timestring(tiv) == time_strings

True

In [21]:
pystare.to_stare_timestring(tiv)

['2021-08-26T17:03:56.626 (48 48) (1)']

### JDs

In [29]:
t = time.Time(now.isoformat(), format='isot')
jd1 = numpy.array([t.jd1])
jd2 = numpy.array([t.jd2])
print(jd1, jd2)

[2459493.] [0.25661248]


In [27]:
tiv = pystare.from_julian_date(jd1, jd2, scale='tai')

In [30]:
pystare.to_julian_date(tiv, scale='tai')

(array([2459215.5]), array([277.75661248]))

### Temporal Triple

In [31]:
triple = pystare.to_temporal_triple_tai(tiv)
triple = pystare.to_temporal_triple_ms(tiv)
triple

(array([2276139850217242877]),
 array([2276139850217271489]),
 array([2276139850217291773]))

In [32]:
pystare.from_temporal_triple(numpy.array(triple).flatten())

array([2276139850217271489])

### Julian TAI

In [33]:
time = datetime.datetime(2021,2,1,12,0)
time_tuple = time.timetuple()

time = erfa.dtf2d("UTC", *time_tuple[0:6])
print(time[0], time[1])

2459246.5 0.5


In [34]:
tiv_tai = pystare.from_julian_date(jd1=numpy.array([time[0]]), 
                                   jd2=numpy.array([time[1]]),
                                   scale='tai')
tiv_tai

array([2275521502176620737])

In [35]:
tiv_utc = pystare.from_julian_date(jd1=numpy.array([time[0]]), 
                                   jd2=numpy.array([time[1]]),
                                   scale='utc')
tiv_utc

array([2275521502797377729])

In [36]:
pystare.to_julian_date(tiv_tai, scale='tai')

(array([2459215.5]), array([31.5]))

In [37]:
pystare.to_julian_date(tiv_utc, scale='utc')

(array([2459215.5]), array([31.5]))

## Bounds

In [38]:
pystare.lower_bound_tai(tiv_tai)
pystare.upper_bound_tai(tiv_tai)
pystare.lower_bound_ms(tiv_utc)
pystare.upper_bound_ms(tiv_utc)

array([2275521502797398013])

## Resolution 

In [39]:
pystare.set_reverse_resolution
pystare.set_forward_resolution
pystare.reverse_resolution
pystare.forward_resolution
pystare.coarsen
pystare.set_temporal_resolutions_from_sorted
pystare.coarsest_resolution_finer_or_equal_ms

<function pystare.temporal.coarsest_resolution_finer_or_equal_ms(ms)>

## Comparison

In [40]:
pystare.cmp_temporal

pystare.temporal_value_intersection_if_overlap
pystare.temporal_value_union_if_overlap
pystare.temporal_overlap_tai
pystare.temporal_overlap
pystare.temporal_contains_instant

<function pystare.temporal.temporal_contains_instant(indices1, indices2)>